# Modelo de rede neural

## Preparando o ambiente

In [3]:
import numpy as np 
import pandas as pd 
from sklearn.preprocessing import StandardScaler, LabelEncoder
from keras.models import Sequential
from keras.layers import Dense, Dropout
import tensorflow
from sklearn.metrics import classification_report, accuracy_score
import keras

In [4]:
SEED = 42
np.random.seed(SEED)
tensorflow.random.set_seed(SEED)

## Carregando dados

In [5]:
train = pd.read_csv("../data/original/train.csv")
test = pd.read_csv("../data/original/test.csv")

Os dados de teste e treino estão sendo colocados aqui para a transformação, mas lembre que as transformações necessárias (título, idade e outras) já foram decididas com os dados de teste antes. Aqui é para facilitar a transformação. Mesmo assim é necessário cuidado.

In [6]:
train['Type'] = 'train'
test['Type'] = 'test'
data = pd.concat([train, test])
data.sample(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Type
257,1149,NaN,3,"Niklasson, Mr. Samuel",male,28.0,0,0,363611,8.050,NaN,S,test
158,1050,NaN,1,"Borebank, Mr. John James",male,42.0,0,0,110489,26.550,D22,S,test
91,983,NaN,3,"Pedersen, Mr. Olaf",male,NaN,0,0,345498,7.775,NaN,S,test
808,809,0.0,2,"Meyer, Mr. August",male,39.0,0,0,248723,13.000,NaN,S,train
304,1196,NaN,3,"McCarthy, Miss. Catherine Katie""""",female,NaN,0,0,383123,7.750,NaN,Q,test


## Transformando dados

O uso para redes neurais exige transformações adicionais dos dados.

### Obtendo título

Semelhante ao que fizemos antes, mas para diminuir a complexidade da rede, reduzindo a quantidade de títulos ao juntar os mais raros.

In [7]:
data['Title'] = data['Name']

for name_string in data['Name']:
    data['Title'] = data['Name'].str.extract('([A-Za-z]+)\.', expand=True)

<>:4: SyntaxWarning: invalid escape sequence '\.'
<>:4: SyntaxWarning: invalid escape sequence '\.'
C:\Users\salatiel.costa\AppData\Local\Temp\ipykernel_91724\4205466781.py:4: SyntaxWarning: invalid escape sequence '\.'
  data['Title'] = data['Name'].str.extract('([A-Za-z]+)\.', expand=True)


In [8]:
data['Title'].value_counts()

Title
Mr          757
Miss        260
Mrs         197
Master       61
Rev           8
Dr            8
Col           4
Mlle          2
Major         2
Ms            2
Lady          1
Sir           1
Mme           1
Don           1
Capt          1
Countess      1
Jonkheer      1
Dona          1
Name: count, dtype: int64

In [9]:
mapping = {'Mlle': 'Miss', 'Ms': 'Miss', 'Mme': 'Mrs', 'Major': 'Other', 
           'Col': 'Other', 'Dr' : 'Other', 'Rev' : 'Other', 'Capt': 'Other', 
           'Jonkheer': 'Royal', 'Sir': 'Royal', 'Lady': 'Royal', 
           'Don': 'Royal', 'Countess': 'Royal', 'Dona': 'Royal'}
data.replace({'Title': mapping}, inplace=True)
data['Title'].value_counts()

Title
Mr        757
Miss      264
Mrs       198
Master     61
Other      23
Royal       6
Name: count, dtype: int64

## Preenchendo idade

Como uma alternativa (que nos testes funcionou melhor), a idade sendo preenchida pelo título. Note como uma ideia leva a outra. Ao reduzir a quantidade de títulos por causa do modelo, levou a ideia de preencher a idade pela mediana nos títulos.

In [10]:
titles = list(data['Title'].unique())
titles

['Mr', 'Mrs', 'Miss', 'Master', 'Royal', 'Other']

In [11]:
for title in titles:
    age_to_impute = data.groupby('Title')['Age'].median()[titles.index(title)]
    data.loc[(data['Age'].isnull()) & (data['Title'] == title), 'Age'] = age_to_impute

C:\Users\salatiel.costa\AppData\Local\Temp\ipykernel_91724\1982622204.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  age_to_impute = data.groupby('Title')['Age'].median()[titles.index(title)]
C:\Users\salatiel.costa\AppData\Local\Temp\ipykernel_91724\1982622204.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  age_to_impute = data.groupby('Title')['Age'].median()[titles.index(title)]
C:\Users\salatiel.costa\AppData\Local\Temp\ipykernel_91724\1982622204.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent

### Refazendo o atributo do tamanho da família

Somando um para considerar a própria pessoa e não deixar ninguém com valor 0. Quando um valor é ZERO ele causa um impacto na rede neural que funciona com multiplicações.

In [12]:
data['Family_Size'] = data['Parch'] + data['SibSp'] + 1

In [13]:
data.loc[:,'FsizeD'] = 'Alone'
data.loc[(data['Family_Size'] > 1),'FsizeD'] = 'Small'
data.loc[(data['Family_Size'] > 4),'FsizeD'] = 'Big'

Note também a redução pelo tamanho da família. O objetivo será reduzir a dimensionalidade.

In [14]:
data

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Type,Title,Family_Size,FsizeD
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,train,Mr,2,Small
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,train,Mrs,2,Small
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,train,Miss,1,Alone
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,train,Mrs,2,Small
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,train,Mr,1,Alone
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,NaN,3,"Spector, Mr. Woolf",male,4.0,0,0,A.5. 3236,8.0500,NaN,S,test,Mr,1,Alone
414,1306,NaN,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C,test,Royal,1,Alone
415,1307,NaN,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S,test,Mr,1,Alone
416,1308,NaN,3,"Ware, Mr. Frederick",male,4.0,0,0,359309,8.0500,NaN,S,test,Mr,1,Alone


### Quando não tiver o preço do ticket

Informação que apenas os dados de teste possuem inválidos. Em um cenário real, teríamos um erro ou descartado o registro, colocado isso em log e adicionado um tratamento genérico. Por isso aqui precisa ser um tratamento o mais genérico possível, como colocar pela mediana da classe.

In [15]:
fa = data[data["Pclass"] == 3]
data['Fare'].fillna(fa['Fare'].median(), inplace = True)

C:\Users\salatiel.costa\AppData\Local\Temp\ipykernel_91724\2742172288.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['Fare'].fillna(fa['Fare'].median(), inplace = True)


### Identificando criança

Como já vimos, a idade tem uma forte influência na sobrevivência. Porém, redes neurais fazem multiplicações e simplesmente considerar a idade pode distorcer o modelo. Mesmo uma "escala" de faixa etária pode gerar problema. Por isso uma flag. Esse tipo de técnica é bastante comum e pode ser usada em vários modelos.

Inclusive, quando testamos um modelo e modificamos a limpeza dos dados por causa dele, pode ser relevante voltar em algoritmos que descartamos e testar com as novas informações.

In [16]:
data.loc[:,'Child'] = 1
data.loc[(data['Age'] >= 18),'Child'] =0

#### Sobrevivência familiar

Uma ideia que encontrei [aqui](https://www.kaggle.com/code/konstantinmasich/titanic-0-82-0-83) levou a considerar outro atributo possível: quantos parentes sobreviveram. Uma das vantagens de competições e estudos com o Kaggle, mesmo as competições de treino ou que já terminaram, é poder consultar as soluções que outras pessoas pensaram.

In [17]:
data['Last_Name'] = data['Name'].apply(lambda x: str.split(x, ",")[0])
DEFAULT_SURVIVAL_VALUE = 0.5

data['Family_Survival'] = DEFAULT_SURVIVAL_VALUE
for grp, grp_df in data[['Survived','Name', 'Last_Name', 'Fare', 'Ticket', 'PassengerId',
                           'SibSp', 'Parch', 'Age', 'Cabin']].groupby(['Last_Name', 'Fare']):
                               
    if (len(grp_df) != 1):
        # A Family group is found.
        for ind, row in grp_df.iterrows():
            smax = grp_df.drop(ind)['Survived'].max()
            smin = grp_df.drop(ind)['Survived'].min()
            passID = row['PassengerId']
            if (smax == 1.0):
                data.loc[data['PassengerId'] == passID, 'Family_Survival'] = 1
            elif (smin == 0.0):
                data.loc[data['PassengerId'] == passID, 'Family_Survival'] = 0

### Limpando features
Objetivo de reduzir dimensionalidade

In [18]:
data = data.drop(columns = ['Age','Cabin','Embarked','Name','Last_Name',
                            'Parch', 'SibSp','Ticket', 'Family_Size'])

### Transformando features em números

In [19]:
target_col = ["Survived"]
id_dataset = ["Type"]

colunas_categoricas = data.nunique()[data.nunique() < 12].keys().tolist()
colunas_numericas = num_cols   = [x for x in data.columns if x not in colunas_categoricas + target_col + id_dataset]
colunas_binarias = data.nunique()[data.nunique() == 2].keys().tolist()
colunas_multiopcoes = [i for i in colunas_categoricas if i not in colunas_binarias]

In [20]:
le = LabelEncoder()

for i in colunas_binarias :
    data[i] = le.fit_transform(data[i])

data = pd.get_dummies(data = data,columns = colunas_multiopcoes)
data

,PassengerId,Survived,Sex,Fare,Type,Child,Pclass_1,Pclass_2,Pclass_3,Title_Master,...,Title_Mr,Title_Mrs,Title_Other,Title_Royal,FsizeD_Alone,FsizeD_Big,FsizeD_Small,Family_Survival_0.0,Family_Survival_0.5,Family_Survival_1.0
0,1,0,1,7.2500,1,0,False,False,True,False,...,True,False,False,False,False,False,True,False,True,False
1,2,1,0,71.2833,1,0,True,False,False,False,...,False,True,False,False,False,False,True,False,True,False
2,3,1,0,7.9250,1,0,False,False,True,False,...,False,False,False,False,True,False,False,False,True,False
3,4,1,0,53.1000,1,0,True,False,False,False,...,False,True,False,False,False,False,True,True,False,False
4,5,0,1,8.0500,1,0,False,False,True,False,...,True,False,False,False,True,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,2,1,8.0500,0,1,False,False,True,False,...,True,False,False,False,True,False,False,False,True,False
414,1306,2,0,108.9000,0,0,True,False,False,False,...,False,False,False,True,True,False,False,False,True,False
415,1307,2,1,7.2500,0,0,False,False,True,False,...,True,False,False,False,True,False,False,False,True,False
416,1308,2,1,8.0500,0,1,False,False,True,False,...,True,False,False,False,True,False,False,False,True,False


#### Normalizando os dados

In [21]:
std = StandardScaler()
scaled = std.fit_transform(data[num_cols])
scaled = pd.DataFrame(scaled,columns = num_cols)
scaled

,PassengerId,Fare
0,-1.730728,-0.503176
1,-1.728082,0.734809
2,-1.725435,-0.490126
3,-1.722789,0.383263
4,-1.720143,-0.487709
...,...,...
1304,1.720143,-0.487709
1305,1.722789,1.462069
1306,1.725435,-0.503176
1307,1.728082,-0.487709


In [22]:
# Salvando original
df_data_og = data.copy()

# Adicionando coluna normalizada
data = data.drop(columns = num_cols,axis = 1)
data = data.merge(scaled,left_index = True,right_index = True,how = "left")
data = data.drop(columns = ['PassengerId'],axis = 1)
data

,Survived,Sex,Type,Child,Pclass_1,Pclass_2,Pclass_3,Title_Master,Title_Miss,Title_Mr,Title_Mrs,Title_Other,Title_Royal,FsizeD_Alone,FsizeD_Big,FsizeD_Small,Family_Survival_0.0,Family_Survival_0.5,Family_Survival_1.0,Fare
0,0,1,1,0,False,False,True,False,False,True,False,False,False,False,False,True,False,True,False,-0.503176
1,1,0,1,0,True,False,False,False,False,False,True,False,False,False,False,True,False,True,False,0.734809
2,1,0,1,0,False,False,True,False,True,False,False,False,False,True,False,False,False,True,False,-0.490126
3,1,0,1,0,True,False,False,False,False,False,True,False,False,False,False,True,True,False,False,0.383263
4,0,1,1,0,False,False,True,False,False,True,False,False,False,True,False,False,False,True,False,-0.487709
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,2,1,0,1,False,False,True,False,False,True,False,False,False,True,False,False,False,True,False,-0.643344
414,2,0,0,0,True,False,False,False,False,False,False,False,True,True,False,False,False,True,False,-0.490126
415,2,1,0,0,False,False,True,False,False,True,False,False,False,True,False,False,False,True,False,-0.487709
416,2,1,0,1,False,False,True,False,False,True,False,False,False,True,False,False,False,True,False,-0.015006


In [23]:
cols = data.columns.tolist()
# Garantindo que a coluna "Survived" é a primeira
cols.insert(0, cols.pop(cols.index('Survived')))

# Reordenando as colunas
data = data.reindex(columns= cols)
data

,Survived,Sex,Type,Child,Pclass_1,Pclass_2,Pclass_3,Title_Master,Title_Miss,Title_Mr,Title_Mrs,Title_Other,Title_Royal,FsizeD_Alone,FsizeD_Big,FsizeD_Small,Family_Survival_0.0,Family_Survival_0.5,Family_Survival_1.0,Fare
0,0,1,1,0,False,False,True,False,False,True,False,False,False,False,False,True,False,True,False,-0.503176
1,1,0,1,0,True,False,False,False,False,False,True,False,False,False,False,True,False,True,False,0.734809
2,1,0,1,0,False,False,True,False,True,False,False,False,False,True,False,False,False,True,False,-0.490126
3,1,0,1,0,True,False,False,False,False,False,True,False,False,False,False,True,True,False,False,0.383263
4,0,1,1,0,False,False,True,False,False,True,False,False,False,True,False,False,False,True,False,-0.487709
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,2,1,0,1,False,False,True,False,False,True,False,False,False,True,False,False,False,True,False,-0.643344
414,2,0,0,0,True,False,False,False,False,False,False,False,True,True,False,False,False,True,False,-0.490126
415,2,1,0,0,False,False,True,False,False,True,False,False,False,True,False,False,False,True,False,-0.487709
416,2,1,0,1,False,False,True,False,False,True,False,False,False,True,False,False,False,True,False,-0.015006


### Separando treino e validação

In [24]:
train = data[data['Type'] == 1].drop(columns = ['Type'])
validation = data[data['Type'] == 0].drop(columns = ['Type'])

In [25]:
X = train.iloc[:, 1:20].values.astype(np.float32)
y = train.iloc[:,0].values.astype(np.float32)

In [26]:
X_validation = validation.iloc[:, 1:20].values.astype(np.float32)

## Modelo rede neural

O processo de decidir qual a melhor rede a se criar pode ser longo e envolver muita coisa. Apenas vamos criar uma rede que pareça interessante, com `relu` na camada oculta e `sigmoid` para a classificação e um `dropout` de $0.2$.

In [27]:
def create_baseline():    
    model = Sequential()
    model.add(Dense(13, input_dim = 18, activation = 'relu'))
    model.add(Dropout(0.1))
    model.add(Dense(16, activation = 'relu'))
    model.add(Dense(1, activation = 'sigmoid'))
    
    model.compile(loss='binary_crossentropy', optimizer = "adam", metrics = ['accuracy'])
    
    return model

### Treinando a rede neural

O treinamento da rede abaixo também pode levar algum tempo.

In [28]:
estimator = create_baseline()
hist = estimator.fit(X, y, epochs = 50)

c:\Python312\Lib\site-packages\keras\src\layers\core\dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50


28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.6218 - loss: 0.6829   
Epoch 2/50
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 579us/step - accuracy: 0.7582 - loss: 0.5821
Epoch 3/50
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8144 - loss: 0.5189 
Epoch 4/50
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 579us/step - accuracy: 0.8276 - loss: 0.4776
Epoch 5/50
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 964us/step - accuracy: 0.8256 - loss: 0.4550
Epoch 6/50
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 889us/step - accuracy: 0.8262 - loss: 0.4378
Epoch 7/50
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 944us/step - accuracy: 0.8407 - loss: 0.4073
Epoch 8/50
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 672us/step - accuracy: 0.8275 - loss: 0.4036
Epoch 9/50
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 730us/step - accuracy: 0.8346 - loss: 0.4064
Epoch 10/50
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 320us/step - accuracy: 0.8487 - loss: 0.3926
Epoch 11/50
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8453 - loss: 0.3988 
Epoch 12/50
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0

## Validando com dados de teste do Kaggle

In [29]:
predictions = ((estimator.predict(X_validation) >= 0.5).T[0] * 1)
predictions


14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 


array([0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0,

In [30]:
ground_truth = pd.read_csv('../data/original/ground_truth.csv')

resultado = pd.DataFrame()
resultado['PassengerId'] = ground_truth['PassengerId']
resultado['Survived'] = predictions

merged = pd.merge(ground_truth, resultado, on='PassengerId', how='inner', suffixes=('_expected', '_predicted'))
acuracia = round(merged[merged['Survived_expected'] == merged['Survived_predicted']].shape[0] / merged.shape[0] * 100, 2)
print(f'Acurácia de {acuracia}% nos dados de teste (submissão Kaggle)')

Acurácia de 80.38% nos dados de teste (submissão Kaggle)


In [31]:
print(classification_report(ground_truth['Survived'], predictions))
print(accuracy_score(ground_truth['Survived'], predictions))

              precision    recall  f1-score   support

           0       0.83      0.86      0.85       260
           1       0.76      0.71      0.73       158

    accuracy                           0.80       418
   macro avg       0.79      0.79      0.79       418
weighted avg       0.80      0.80      0.80       418

0.8038277511961722


In [32]:
resultado.to_csv("../data/submissions/4_keras_ex_nn.csv", index = False)

Atingimos uma acurácia maior do que a Decision Tree, mas ainda há espaço para melhoria. O que fizemos até aqui foi com o objetivo didático.